## KerasTuner


In [91]:
import pandas as pd 
import numpy as np 

In [92]:
df=pd.read_csv("diabetes.csv")

In [93]:
df.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0


In [94]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [95]:
X = df.iloc[:,0:-1].values
Y = df.iloc[:,-1].values

In [96]:
# X
# Y

Scaling X & Y 

In [97]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()

In [98]:
X = scalar.fit_transform(X)

In [99]:
# X
# X.shape

In [100]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=1) 

In [101]:
import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [102]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=['accuracy'])

In [103]:
model.fit(x_train,y_train, batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 2s 29ms/step - loss: 0.6611 - accuracy: 0.6433 - val_loss: 0.6426 - val_accuracy: 0.6169
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6108 - accuracy: 0.6792 - val_loss: 0.5978 - val_accuracy: 0.6623
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5750 - accuracy: 0.6938 - val_loss: 0.5671 - val_accuracy: 0.7013
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5482 - accuracy: 0.7134 - val_loss: 0.5453 - val_accuracy: 0.7208
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5287 - accuracy: 0.7215 - val_loss: 0.5307 - val_accuracy: 0.7078
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5134 - accuracy: 0.7280 - val_loss: 0.5196 - val_accuracy: 0.7208
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5018 - accuracy: 0.7329 - val_loss: 0.5118 - val_accuracy: 0.7208
Epoch 8/100


## Hyperparameter Tuning via Keras-Tuner

#### 1. How to Select Appropriate Optimizer
#### 2. No. of Neurons/Nodes in a Layer 
#### 3. How to Select No. of Layers 
#### 4. All in all 1 Model.

## 1. FInding Best Optimizer 

In [104]:
# !pip install keras-tuner

In [105]:
import kerastuner as kt

hp means hyperparameter

In [106]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    # model.compile(optimizer=hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta']) )
    optimizer_choice = hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer_choice,loss='binary_crossentropy',metrics = ['accuracy'])

    return model 

#### Now we have to Make Tuner Object 
tuner.RandomSearch(build_model,yahape apna objective batao , yahape max Trials )


In [107]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory="mydir",project_name="best_Optimizer") #


Reloading Tuner from mydir\best_Optimizer\tuner0.json


In [108]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test),verbose=2)

In [109]:
tuner.results_summary()

Results summary
Results in mydir\best_Optimizer
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7402597665786743

Trial 3 summary
Hyperparameters:
optimizer: sgd
Score: 0.7337662577629089

Trial 2 summary
Hyperparameters:
optimizer: adam
Score: 0.7272727489471436

Trial 1 summary
Hyperparameters:
optimizer: adadelta
Score: 0.4285714328289032


rmsprop giving best Result 


In [110]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [111]:
best_Model = tuner.get_best_models(num_models=1)[0]

In [112]:
best_Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [113]:
best_Model.fit(x_train,y_train,batch_size=32, epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.5599 - accuracy: 0.7345 - val_loss: 0.5712 - val_accuracy: 0.7468
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5384 - accuracy: 0.7427 - val_loss: 0.5513 - val_accuracy: 0.7532
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5208 - accuracy: 0.7508 - val_loss: 0.5352 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5080 - accuracy: 0.7557 - val_loss: 0.5230 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4980 - accuracy: 0.7655 - val_loss: 0.5146 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4899 - accuracy: 0.7704 - val_loss: 0.5076 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4828 - accuracy: 0.7769 - val_loss: 0.5023 - val_accuracy: 0.7857
Epoch 14

Earlier Optimizer was Adam --> Accuracy was 79.something  
Now Optimizer is RMSProp --> Accuracy is 81 close to 82 

## 2. Finding Right Number of Neurons in the layer 

In [114]:
def build_model(hp):
    model = Sequential() 
    

    units = hp.Int('units',min_value=8,max_value=128)  # lowerlimit,upperlimit 
    model.add(Dense(units=units,activation='relu',input_dim=8))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model 


In [115]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory="mydir",project_name="no_of_neurons_in_layers")

Reloading Tuner from mydir\no_of_neurons_in_layers\tuner0.json


In [116]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test),verbose=2)

In [117]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [118]:
model = tuner.get_best_models(num_models=1)[0]

In [119]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 3ms/step - loss: 0.5066 - accuracy: 0.7508
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4878 - accuracy: 0.7655
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4753 - accuracy: 0.7720
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4667 - accuracy: 0.7736
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4622 - accuracy: 0.7687
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4571 - accuracy: 0.7785
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4543 - accuracy: 0.7752
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4522 - accuracy: 0.7752
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4492 - accuracy: 0.7752
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4480 - accuracy: 0.7785
Epo

## 3. How to Select Number of Layers 

In [120]:
def build_model(hp):
    
    model = Sequential() 
    model.add(Dense(72,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model 

In [121]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=1,directory='mydir',project_name='num_of_layers')

Reloading Tuner from mydir\num_of_layers\tuner0.json


In [122]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [123]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [124]:
model = tuner.get_best_models(num_models=1)[0]
model

In [125]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 2s 4ms/step - loss: 0.4626 - accuracy: 0.7752
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4488 - accuracy: 0.7801
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4458 - accuracy: 0.7997
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4386 - accuracy: 0.7948
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4206 - accuracy: 0.8046
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4216 - accuracy: 0.7948
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4040 - accuracy: 0.8176
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3922 - accuracy: 0.8257
Epoch 15/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3863 - accuracy: 0.8176
Epoch 16/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3822 - accuracy: 0.8404
Epo

Accuracy gone to 99% 

### Everything in 1 Go 

In [126]:
def build_model(hp):

    model = Sequential() 

    counter = 0 
    # units = hp.Int()

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter ==0:
              model.add(
                   Dense(hp.Int('units'+str(i), min_value=8, max_value=120, step = 8),
                        activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                            input_dim=8 ),
                        )
              model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
                        
        else: 
                 model.add(
                   Dense(
                        hp.Int('units'+str(i), min_value=8, max_value=120, step = 8),
                               activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                               )
                        )
                 model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter +=1 
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adamdelta']),
                  loss='binary_crossentropy',metrics=['accuracy'])
    
    return model 

In [127]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='mydir',
    project_name='Final_overall_Best'
    )

Reloading Tuner from mydir\Final_overall_Best\tuner0.json


In [128]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [132]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 24,
 'activation0': 'sigmoid',
 'optimizer': 'nadam',
 'units1': 96,
 'activation1': 'tanh',
 'units2': 64,
 'activation2': 'tanh',
 'units3': 56,
 'activation3': 'tanh',
 'units4': 40,
 'activation4': 'relu',
 'units5': 88,
 'activation5': 'tanh',
 'units6': 120,
 'activation6': 'tanh',
 'dropout0': 0.1,
 'dropout1': 0.1,
 'dropout2': 0.1,
 'dropout3': 0.1,
 'dropout4': 0.1,
 'dropout5': 0.1,
 'dropout6': 0.1}

In [133]:
model = tuner.get_best_models(num_models=1)[0]

In [135]:
model.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test),batch_size=32)

Epoch 6/200
20/20 [==============================] - 0s 12ms/step - loss: 0.4563 - accuracy: 0.7834 - val_loss: 0.4771 - val_accuracy: 0.7727
Epoch 7/200
20/20 [==============================] - 0s 8ms/step - loss: 0.4604 - accuracy: 0.7769 - val_loss: 0.4765 - val_accuracy: 0.7662
Epoch 8/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4483 - accuracy: 0.7736 - val_loss: 0.4809 - val_accuracy: 0.7987
Epoch 9/200
20/20 [==============================] - 0s 8ms/step - loss: 0.4600 - accuracy: 0.7752 - val_loss: 0.4832 - val_accuracy: 0.7727
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4561 - accuracy: 0.7834 - val_loss: 0.4822 - val_accuracy: 0.7727
Epoch 11/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4695 - accuracy: 0.7752 - val_loss: 0.4802 - val_accuracy: 0.7727
Epoch 12/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4545 - accuracy: 0.7785 - val_loss: 0.4732 - val_accuracy: 0.7857
Epoch 13/